In [ ]:
import logging
import os
import time
import lotr

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s: %(message)s')

In [ ]:
timestamp = time.time()

with open(lotr.RUN_BEFORE_SE_STARTED_PATH, 'w', encoding='utf-8'):
    pass

if os.path.exists(lotr.PROJECT_CREATED_PATH):
    os.remove(lotr.PROJECT_CREATED_PATH)

conf = lotr.read_conf()

if (not conf['reprocess_all'] and conf['reprocess_all_on_error'] and
        os.path.exists(lotr.PIPELINE_STARTED_PATH)):
    conf['reprocess_all'] = True
    logging.info('The previous update did not succeed, setting "reprocess_all" to "true" for this run')

lotr.download_sheet(conf)

with open(lotr.PIPELINE_STARTED_PATH, 'w', encoding='utf-8'):
    pass

lotr.extract_data(conf)
sets = lotr.get_sets(conf)
sets = lotr.sanity_check(conf, sets)
lotr.save_data_for_bot(conf)

if conf['output_languages']:
    lotr.verify_images(conf)
    lotr.reset_project_folders(conf)

eons = False
changes = False
for set_id, set_name in sets:
    scratch = set_id in lotr.FOUND_SCRATCH_SETS
    if conf['octgn_set_xml']:
        lotr.generate_octgn_set_xml(conf, set_id, set_name)

    if conf['octgn_o8d']:
        lotr.generate_octgn_o8d(conf, set_id, set_name)

    if conf['ringsdb_csv']:
        lotr.generate_ringsdb_csv(conf, set_id, set_name)

    if conf['dragncards_json']:
        lotr.generate_dragncards_json(conf, set_id, set_name)

    if conf['frenchdb_csv']:
        lotr.generate_frenchdb_csv(conf, set_id, set_name)

    if conf['spanishdb_csv']:
        lotr.generate_spanishdb_csv(conf, set_id, set_name)

    if conf['hallofbeorn_json']:
        for lang in (conf['output_languages'] or ['English']):
            if scratch and lang != 'English':
                continue

            lotr.generate_hallofbeorn_json(conf, set_id, set_name, lang)

    if conf['output_languages']:
        lotr.copy_custom_images(conf, set_id, set_name)

    scratch = set_id in lotr.FOUND_SCRATCH_SETS
    for lang in conf['output_languages']:
        if scratch and lang != 'English':
            continue

        eons = True
        lotr.generate_xml(conf, set_id, set_name, lang)
        lotr.update_xml(conf, set_id, set_name, lang)
        file_changes, _ = lotr.calculate_hashes(set_id, set_name, lang)
        if file_changes:
            changes = True

        lotr.copy_raw_images(conf, set_id, set_name, lang)
        lotr.copy_xml(set_id, set_name, lang)

if conf['octgn_set_xml'] or conf['octgn_o8d']:
    lotr.copy_octgn_outputs(conf, sets)

if conf['ringsdb_csv'] and conf['update_ringsdb']:
    lotr.update_ringsdb(conf, sets)

if changes:
    lotr.create_project()
    with open(lotr.PROJECT_CREATED_PATH, 'w', encoding='utf-8'):
        pass
else:
    if os.path.exists(lotr.PIPELINE_STARTED_PATH):
        os.remove(lotr.PIPELINE_STARTED_PATH)

    if eons:
        logging.info('No changes since the last run, skipping creating the project')
    else:
        logging.info('No project outputs, skipping creating the project')

if os.path.exists(lotr.RUN_BEFORE_SE_STARTED_PATH):
    os.remove(lotr.RUN_BEFORE_SE_STARTED_PATH)

logging.info('Done (%ss)', round(time.time() - timestamp, 3))

### Manual Part:

1. Open `setGenerator.seproject`.
2. Run `Script/makeCards` script by double clicking it.
3. Once completed, close the program (wait until it finished packing the project).

In [ ]:
timestamp = time.time()

if os.path.exists(lotr.PROJECT_PATH):
    actual_sets = lotr.get_actual_sets()
else:
    actual_sets = []

for set_id, set_name in sets:
    scratch = set_id in lotr.FOUND_SCRATCH_SETS
    for lang in conf['output_languages']:
        if scratch and lang != 'English':
            continue

        skip_set, skip_ids = lotr.get_skip_info(set_id, lang)
        if skip_set:
            logging.info('[%s, %s] No changes since the last run,'
                         ' skipping', set_name, lang)
            continue

        if (set_id, lang) not in actual_sets:
            logging.error('[%s, %s] Not found in the project file,'
                          ' skipping', set_name, lang)
            continue

        card_data = lotr.translated_data(set_id, lang)
        card_dict = lotr.full_card_dict()

        if conf['nobleed_300'][lang]:
            lotr.generate_png300_nobleed(conf, set_id, set_name, lang, skip_ids)

        if conf['nobleed_480'][lang]:
            lotr.generate_png480_nobleed(conf, set_id, set_name, lang, skip_ids)

        if conf['nobleed_800'][lang]:
            lotr.generate_png800_nobleed(conf, set_id, set_name, lang, skip_ids)

        if 'db' in conf['outputs'][lang]:
            lotr.generate_png300_db(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_db(conf, set_id, set_name, lang, card_data)

        if 'dragncards_hq' in conf['outputs'][lang]:
            lotr.generate_png480_dragncards_hq(set_id, set_name, lang, skip_ids)
            lotr.generate_dragncards_hq(conf, set_id, set_name, lang, card_data)

        if 'octgn' in conf['outputs'][lang]:
            lotr.generate_png300_octgn(set_id, set_name, lang, skip_ids)
            lotr.generate_octgn(conf, set_id, set_name, lang, card_data)

        if 'rules_pdf' in conf['outputs'][lang]:
            lotr.generate_png300_rules_pdf(set_id, set_name, lang, skip_ids, card_data)
            lotr.generate_rules_pdf(conf, set_id, set_name, lang)

        if 'pdf' in conf['outputs'][lang]:
            lotr.generate_png300_pdf(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_pdf(conf, set_id, set_name, lang, card_data)

        if 'genericpng_pdf' in conf['outputs'][lang]:
            lotr.generate_png800_pdf(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_genericpng_pdf(conf, set_id, set_name, lang, card_data)

        if 'makeplayingcards' in conf['outputs'][lang]:
            lotr.generate_png800_bleedmpc(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_mpc(conf, set_id, set_name, lang, card_data)

        if 'drivethrucards' in conf['outputs'][lang]:
            lotr.generate_jpg300_bleeddtc(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_dtc(conf, set_id, set_name, lang, card_data)

        if 'mbprint' in conf['outputs'][lang]:
            lotr.generate_jpg800_bleedmbprint(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_mbprint(conf, set_id, set_name, lang, card_data)

        if 'genericpng' in conf['outputs'][lang]:
            lotr.generate_png800_bleedgeneric(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_genericpng(conf, set_id, set_name, lang, card_data)

        if 'tts' in conf['outputs'][lang]:
            lotr.generate_tts(conf, set_id, set_name, lang, card_dict, scratch)

if 'English' in conf['output_languages']:
    updated_sets = [s for s in sets if (s[0], 'English') in actual_sets
                    and not lotr.get_skip_info(s[0], 'English')[0]]
else:
    updated_sets = []

if (conf['db_destination_path'] and 'English' in conf['output_languages'] and
        'db' in conf['outputs']['English'] and updated_sets):
    lotr.copy_db_outputs(conf, updated_sets)

if (conf['octgn_image_destination_path'] and 'English' in conf['output_languages'] and
        'octgn' in conf['outputs']['English'] and updated_sets):
    lotr.copy_octgn_image_outputs(conf, updated_sets)

if (conf['tts_destination_path'] and 'English' in conf['output_languages'] and
        'tts' in conf['outputs']['English'] and updated_sets):
    lotr.copy_tts_outputs(conf, updated_sets)

if conf['upload_dragncards'] and conf['dragncards_hostname'] and conf['dragncards_id_rsa_path']:
    lotr.upload_dragncards(conf, sets, updated_sets)

if os.path.exists(lotr.PIPELINE_STARTED_PATH):
    os.remove(lotr.PIPELINE_STARTED_PATH)

logging.info('Done (%ss)', round(time.time() - timestamp, 3))